In [1]:
from pyfinite import ffield
field = ffield.FField(7, gen=0x83, useLUT=-1)

exponents = [[] for base in range(128)]
for base in range(128):
    exponents[base].append(1)
    
for base in range(128):
    for exp in range(1,127):
        result = field.Multiply(exponents[base][exp-1], base)
        exponents[base].append(result)
        
num_sets = 3

# cipher texts when a single byte is active and equal to 'hk' = 37, 'lu' = 111, and 'mt' = 126
active_byte = [37, 111, 126]
byte_active_ciphertext = [[] for i in range(num_sets)]
byte_active_ciphertext[0] = ["lmgmiiljkpktjmko", "ffgtlslsfflsinku", "ffffljhljijnjlkl", "ffffffjijljnfqjj",
                         "fffffffflsgqmtfh", "ffffffffffftmlhf", "ffffffffffffkplr", "fffffffffffffffl"]
byte_active_ciphertext[1] = ["llgompiljrjkgsli", "ffhlkrktlnglfqkf", "ffffksjfktjtfpik", "ffffffitfpfsisik",
                          "ffffffffhghkmlhp", "fffffffffflrlojt", "ffffffffffffmqlu", "ffffffffffffffig"]
byte_active_ciphertext[2] = ["ljjpimmtgofuhtkr", "ffirgljujkimljgl", "fffflsfsgflplnij", "ffffffifkflilgjl",
                          "ffffffffhjmthogt", "ffffffffffgsimlp", "ffffffffffffmhhh", "ffffffffffffffkf"]

cipher_bytes = [[] for set in range(num_sets)]

for byte in range(8):
    for cset in range(num_sets):
        nibble1 = byte_active_ciphertext[cset][byte][2*byte]
        nibble2 = byte_active_ciphertext[cset][byte][2*byte+1]
        cipher_byte = (ord(nibble1) - ord('f'))*16 + (ord(nibble2) - ord('f'))
        cipher_bytes[cset].append(cipher_byte)

# determine candidates for the exponent vector and corresponding diagonal elements of the A matrix
# key: candidate exponent-values, value: candidate diagonal-element for corresponding byte positions
candidate_pairs = [{} for byte_pos in range(8)]

for byte in range(8):
    for diag_element in range(128):
        for exp in range(1,127):
            eaeae = []
            for cset in range(num_sets): # don't need to use all three sets
                ea = field.Multiply(exponents[active_byte[cset]][exp], diag_element)
                eaea = field.Multiply(exponents[ea][exp], diag_element)
                eaeae.append(exponents[eaea][exp])
            
            for cset in range(num_sets):
                if eaeae[cset] != cipher_bytes[cset][byte]:
                    break
                if cset == num_sets - 1:
                    candidate_pairs[byte][exp] = diag_element

for byte in range(8):
    print(candidate_pairs[byte])

{18: 96, 21: 104, 88: 126}
{11: 13, 82: 64, 34: 85}
{20: 4, 108: 97}
{29: 11, 55: 14, 43: 20}
{26: 78, 113: 121, 115: 122}
{54: 73, 10: 83, 63: 85}
{102: 53, 33: 95, 119: 124}
{56: 28, 48: 43, 23: 50}


In [2]:
import numpy as np

cipher_bytes = [[] for set in range(num_sets)]

for byte in range(7):
    for cset in range(num_sets):
        nibble1 = byte_active_ciphertext[cset][byte][2*byte+2]
        nibble2 = byte_active_ciphertext[cset][byte][2*byte+3]
        cipher_byte = (ord(nibble1) - ord('f'))*16 + (ord(nibble2) - ord('f'))
        cipher_bytes[cset].append(cipher_byte)

A = np.zeros((8, 8), dtype = int)
E = np.zeros((8), dtype = int)
# determine candidates for elements a_{i-1, i} in the A matrix
# using the candidates elements for E and the diagonal values for A
num_sets = 3
for element_pos in range(7):
    for _, (exp_l, d_l) in enumerate(candidate_pairs[element_pos].items()):
        for _, (exp_h, d_h) in enumerate(candidate_pairs[element_pos+1].items()):
            for a_hl in range(128):
                for text_set in range(num_sets):
                    E_l = exponents[active_byte[text_set]][exp_l]
                    
                    EA_l = field.Multiply(d_l, E_l)
                    EA_h = field.Multiply(a_hl, E_l)

                    EAE_l = exponents[EA_l][exp_l]
                    EAE_h = exponents[EA_h][exp_h]

                    term1 = field.Multiply(EAE_l, a_hl)
                    term2 = field.Multiply(EAE_h, d_h)
                    EAEA_h = field.Add(term1, term2)

                    EAEAE_h = exponents[EAEA_h][exp_h]

                    if EAEAE_h != cipher_bytes[text_set][element_pos]:
                        break
                    if text_set == num_sets - 1:
                        A[element_pos][element_pos], A[element_pos+1][element_pos] = d_l, a_hl
                        E[element_pos], E[element_pos+1] = exp_l,exp_h
A[element_pos+1][element_pos+1] = d_h 

print("A = \n{}\n E = {}".format(A, E))

A = 
[[126   0   0   0   0   0   0   0]
 [ 32  85   0   0   0   0   0   0]
 [  0 100   4   0   0   0   0   0]
 [  0   0  29  20   0   0   0   0]
 [  0   0   0  17  78   0   0   0]
 [  0   0   0   0  94  85   0   0]
 [  0   0   0   0   0  77  53   0]
 [  0   0   0   0   0   0  62  50]]
 E = [ 88  34  20  43  26  63 102  23]


In [16]:
for diagonal in range(2,8):
    r, c = diagonal, 0
    for col in range(8-diagonal):
        for a in range(128):
            A[r][c] = a
            flag = 0
            for text_set in range(num_sets):
                total = 0
                for ind in range(c,r+1):
                    total = field.Add(total,field.Multiply(exponents[field.Multiply(exponents[active_byte[text_set]][E[c]],A[ind][c])][E[ind]],A[r][ind]))
                total = exponents[total][E[r]]
                
                #compute corresponding ciphertext
                nibble1 = byte_active_ciphertext[text_set][c][2*r]
                nibble2 = byte_active_ciphertext[text_set][c][2*r+1]
                cipher_byte = (ord(nibble1) - ord('f'))*16 + (ord(nibble2) - ord('f'))
                
                if total != cipher_byte:
                    flag = 1
                    break
            if flag == 0:
                A[r][c] = a
                break
        r += 1
        c += 1
print("A = \n{}\n E = {}".format(A, E))

A = 
[[126   0   0   0   0   0   0   0]
 [ 32  85   0   0   0   0   0   0]
 [ 68 100   4   0   0   0   0   0]
 [126  91  29  20   0   0   0   0]
 [ 74  83 122  17  78   0   0   0]
 [ 76  99  69   0  94  85   0   0]
 [ 58 111  40  93 114  77  53   0]
 [ 39 122 108  68  78   2  62  50]]
 E = [ 88  34  20  43  26  63 102  23]


In [51]:
import numpy as np

block_size = 8

invE = np.zeros((128, 128), dtype = int)
for base in range(128):
    for exp in range(1,127):
        invE[exp][exponents[base][exp]] = base

inverses = [1] # dummy inverse of 0
for element in range(1, 128):
    inverses.append(field.Inverse(element))
    assert field.Multiply(element, inverses[element]) == 1
    
A = np.array((A))

augmentedA = np.zeros((block_size, block_size*2), dtype = int)
for row in range(block_size):
    for col in range(block_size):
        augmentedA[row][col] = A[row][col]
    augmentedA[row][col+row+1] = 1

for row in range(block_size):
    assert np.any(augmentedA[row:,row] != 0) # assert pivot row exists: A is invertible
    pivot_row = np.where(augmentedA[row:,row] != 0)[0][0] + row
    augmentedA[[row, pivot_row]] = augmentedA[[pivot_row, row]]
    # make pivot element 1
    mul_fact = inverses[augmentedA[row][row]]
    for element in range(block_size*2):
        augmentedA[row][element] = field.Multiply(augmentedA[row][element], mul_fact)
    for index in range(block_size):
        if index == row:
            continue
        if augmentedA[index][row] != 0:
            mult_fact = augmentedA[index][row]
            for element in range(block_size*2):
                augmentedA[index][element] = field.Add(field.Multiply(augmentedA[row][element], mult_fact), augmentedA[index][element])

inverseA = np.zeros((block_size, block_size), dtype = int)
for row in range(block_size):
    for col in range(block_size):
        inverseA[row][col] = augmentedA[row][block_size+col]
print("A inverse: \n{}".format(inverseA))

A inverse 
[[  3   0   0   0   0   0   0   0]
 [ 23 109   0   0   0   0   0   0]
 [105  72  97   0   0   0   0   0]
 [ 55 104  70  75   0   0   0   0]
 [122 119  55 104  95   0   0   0]
 [116  76  65  85  68 109   0   0]
 [ 86  31  40  88  28  94  81   0]
 [ 26  88 122  10  93  55  63  80]]


In [48]:
password = "hjhmmjhmmoloigjqhgkhgqltjsihjqfi"
block_size = 16
num_blocks = int(len(password) / block_size)

def invE_transform(block, E):
    transformed = []
    for byte in range(8):
        transformed.append(invE[E[byte]][block[byte]])
    return transformed

def invA_transform(block, A):
    transformed = []
    for row_num in range(8):
        elem_sum = 0
        for col_num in range(8):
            elem = field.Multiply(A[row_num][col_num], block[col_num])
            elem_sum = field.Add(elem, elem_sum)
        transformed.append(elem_sum)
    return transformed

decoded = ""
for block_num in range(num_blocks):
    block_char = password[block_size*block_num:block_size*(block_num+1)]
    block = []
    for byte in range(8):
        nibble1 = block_char[2*byte]
        nibble2 = block_char[2*byte+1]
        block.append((ord(nibble1) - ord('f'))*16 + (ord(nibble2) - ord('f')))
        
    Einverted = invE_transform(block, E)
    EA = invE_transform(invA_transform(Einverted, inverseA), E)
    EAEAE = invE_transform(invA_transform(EA, inverseA), E)
    print(EAEAE)
    for char in EAEAE:
        decoded += chr(char)
print(decoded)
    

[99, 103, 98, 104, 106, 109, 98, 105]
[120, 101, 98, 115, 106, 99, 118, 114]
cgbhjmbixebsjcvr
